In [38]:
from langchain.llms import Ollama
from langchain.chat_models import ChatOllama

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain.tools import DuckDuckGoSearchRun, BaseTool

from langchain.agents import initialize_agent, AgentType, tool, Tool, load_tools
from langchain.tools.render import format_tool_to_openai_function

from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMMathChain

In [39]:
llm = Ollama(model="llama2:13b")

In [40]:
class WordLengthTool(BaseTool):
    name = "Word Length Tool"
    description = "Use this tool when you need to find the length of a given word"

    def _run(self, word: str):
        return str(len(word))

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")

In [41]:
tools = [WordLengthTool()]

In [64]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True, output_key="output"
)

In [65]:
agent = initialize_agent(
    agent="chat-conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    early_stopping_method="generate",
    memory=memory,
    # handle_parsing_errors=True,
    #agent_kwargs={"output_parser": parser}
)

In [66]:
agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain.schema.messages.AIMessage, langchain.schema.messages.HumanMessage, langchain.schema.messages.ChatMessage, langchain.schema.messages.SystemMessage, langchain.schema.messages.FunctionMessage, langchain.schema.messages.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain.schema.messages.AIMessage, langchain.schema.messages.HumanMessage, langchain.schema.messages.ChatMessage, langchain.schema.messages.SystemMessage, langchain.schema.messages.FunctionMessage, langchain.schema.messages.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a la

In [67]:
# View the current System Prompt Template
print(agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

In [68]:
# View the current Human Message Prompt Template
print(agent.agent.llm_chain.prompt.messages[2].prompt.template)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Word Length Tool: Use this tool when you need to find the length of a given word

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string, \\ The action to take. Must be one of Word Length Tool
    "action_input": string \\ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \\ You should put what you want to return to use here
}}
```

USER'S INPUT
--------------------
Here is the user's input (remember to respond with a 

In [69]:
# Special llama2 tokens that it was trained on.
B_INST, E_INST = "[INST]", "[/INST]" # Begin and end an instruction
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n" #Begin and end system prompt

In [70]:
# Define a new system prompt
sys_msg = B_SYS + """You are Assistant. Assistant is a expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to respond to the User and use tools using JSON strings that contain "action" and "action_input" parameters.

All of Assistant's communication is performed using this JSON format. The assistant NEVER outputs anything other than a json object with an action and action_input fields!

Assistant can also use tools by responding to the user with tool use instructions in the same "action" and "action_input" JSON format. Tools available to Assistant are:

- "Word Length Tool": Useful for when you need to get the length of a word.
  - To use the Word Length Tool, Assistant should write like so:
    ```json
    {{"action": "Word Length Tool",
      "action_input": "elephant"}}
    ```

Here is an example of a previous conversation between User and Assistant:
---
User: Hey how are you today?
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "I'm good thanks, how are you?"}}
```
User: I'm great, what is the length of the word educate?
Assistant: ```json
{{"action": "Word Length Tool",
 "action_input": "educate"}}
```
User: 7
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 7!"}}
```
User: Thanks could you tell me what the length of "turtle" is?
Assistant: ```json
{{"action": "Word Length Tool",
 "action_input": "turtle"}}
```
User: 6
Assistant: ```json
{{"action": "Final Answer",
 "action_input": "It looks like the answer is 6!"}}
```
---
Notice that after Assistant uses a tool, User will give the output of that tool. Then this output can be returned as a final answer.
Assistant will only use the available tools and NEVER a tool not listed. If the User's question does not require the use of a tool, Assistant will use the "Final Answer" action to give a normal response.
""" + E_SYS
new_prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
agent.agent.llm_chain.prompt = new_prompt

In [58]:
# define a new user prompt template
instruction = B_INST + " Respond to the following in JSON with 'action' and 'action_input' values. Output NOTHING else!" + E_INST
human_msg = instruction + "\nUser: {input}"

agent.agent.llm_chain.prompt.messages[2].prompt.template = human_msg

In [72]:
agent("Hello, how are you?")



> Entering new AgentExecutor chain...

{
"action": "Final Answer",
"action_input": "I'm good, thanks for asking!"
}

> Finished chain.


{'input': 'Hello, how are you?',
 'chat_history': [],
 'output': "I'm good, thanks for asking!"}

In [73]:
agent("What is the length of the word turtle")



> Entering new AgentExecutor chain...

{
"action": "Word Length Tool",
"action_input": "turtle"}
Observation: 6
Thought:{
"action": "Final Answer",
"action_input": "It looks like the answer is 6!"
}

> Finished chain.


{'input': 'What is the length of the word turtle',
 'chat_history': [HumanMessage(content='Hello, how are you?'),
  AIMessage(content="I'm good, thanks for asking!")],
 'output': 'It looks like the answer is 6!'}

In [74]:
agent("What is the length of the word 'educa'?")



> Entering new AgentExecutor chain...

Assistant: ```json
{"action": "Word Length Tool",
 "action_input": "educa"}
```
Observation: 5
Thought:
{
"action": "Final Answer",
"action_input": "It looks like the answer is 5!"
}

> Finished chain.


{'input': "What is the length of the word 'educa'?",
 'chat_history': [HumanMessage(content='Hello, how are you?'),
  AIMessage(content="I'm good, thanks for asking!"),
  HumanMessage(content='What is the length of the word turtle'),
  AIMessage(content='It looks like the answer is 6!')],
 'output': 'It looks like the answer is 5!'}

In [75]:
agent("Can you write a short poem about canada?")



> Entering new AgentExecutor chain...

{
"action": "Final Answer",
"action_input": """
A land of maple syrup and mounties so brave,
Canada's beauty is truly a sight to crave.
From the Rocky Mountains to the Great Lakes,
This country's splendor leaves all in awe.
"""
}

> Finished chain.


{'input': 'Can you write a short poem about canada?',
 'chat_history': [HumanMessage(content='Hello, how are you?'),
  AIMessage(content="I'm good, thanks for asking!"),
  HumanMessage(content='What is the length of the word turtle'),
  AIMessage(content='It looks like the answer is 6!'),
  HumanMessage(content="What is the length of the word 'educa'?"),
  AIMessage(content='It looks like the answer is 5!')],
 'output': '""\nA land of maple syrup and mounties so brave,\nCanada\'s beauty is truly a sight to crave.\nFrom the Rocky Mountains to the Great Lakes,\nThis country\'s splendor leaves all in awe.\n""'}

In [76]:
agent("How many letters are in this string: 'slkjf92j'")



> Entering new AgentExecutor chain...

Assistant: ```json
{"action": "Word Length Tool",
 "action_input": "slkjf92j"}
```
Observation: 8
Thought:{
"action": "Final Answer",
"action_input": "8"
}

> Finished chain.


{'input': "How many letters are in this string: 'slkjf92j'",
 'chat_history': [HumanMessage(content='Hello, how are you?'),
  AIMessage(content="I'm good, thanks for asking!"),
  HumanMessage(content='What is the length of the word turtle'),
  AIMessage(content='It looks like the answer is 6!'),
  HumanMessage(content="What is the length of the word 'educa'?"),
  AIMessage(content='It looks like the answer is 5!'),
  HumanMessage(content='Can you write a short poem about canada?'),
  AIMessage(content='""\nA land of maple syrup and mounties so brave,\nCanada\'s beauty is truly a sight to crave.\nFrom the Rocky Mountains to the Great Lakes,\nThis country\'s splendor leaves all in awe.\n""')],
 'output': '8'}

In [77]:
llm("How many letters are in this string: 'slkjf92j'")

'\nThe string "slkjf92j" contains 9 letters.'